In [1]:
h5_file = 'U19-depth.h5'

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12876164598493181299
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6574322484
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7357746677154168551
physical_device_desc: "device: 0, name: GeForce RTX 2080 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5"
]


In [3]:
from keras.models import load_model

model = load_model('lrcn4_50e.h5')

Using TensorFlow backend.




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where












In [4]:
import h5py

hf = h5py.File(h5_file,'r')
tags = [n.decode('unicode_escape') for n in hf['Tags']]
tag_indices = [[],[],[],[],[]]
for i in range(len(tags)):
    # Switch-case would be appropriate here
    if tags[i] == 'dart':
        tag_indices[0].append(i)
    elif tags[i] == 'micro':
        tag_indices[1].append(i)
    elif tags[i] == 'pause':
        tag_indices[2].append(i)
    elif tags[i] == 'rear':
        tag_indices[3].append(i)
    else:
        tag_indices[4].append(i)
hf.close()

In [5]:
print(len(tags))
for i in tag_indices:
    print(len(i))

79719
9158
11858
16804
34267
7632


In [6]:
hf = h5py.File(h5_file,'r')
for x in tag_indices:
    print(hf['Tags'][x[0]])
hf.close()

b'dart'
b'micro'
b'pause'
b'rear'
b'walk'


In [7]:
import numpy as np

def fill_instances(indices,file):
    instances = np.zeros((len(indices),30,80,80,1),np.int8)
    hf = h5py.File(file,'r')
    for i in range(len(indices)):
        instances[i] = hf['Instances'][indices[i]]
    hf.close()
    return instances

In [8]:
import numpy as np

dart_instances = fill_instances(tag_indices[0],h5_file)
micro_instances = fill_instances(tag_indices[1],h5_file)
pause_instances = fill_instances(tag_indices[2],h5_file)
rear_instances = fill_instances(tag_indices[3],h5_file)
walk_instances = fill_instances(tag_indices[4],h5_file)

In [9]:
dart_pred = model.predict(dart_instances)
micro_pred = model.predict(micro_instances)
pause_pred = model.predict(pause_instances)
rear_pred = model.predict(rear_instances)
walk_pred = model.predict(walk_instances)

In [10]:
def get_stats(pred):
    stats = np.array([0,0,0,0,0])
    for i in pred:
        m_index = np.where(i == np.amax(i))
        stats[m_index[0][0]] += 1
    return stats

In [11]:
dart_stats = get_stats(dart_pred)
print(dart_stats/np.sum(dart_stats))
print(np.sum(dart_stats))

[0.52675257 0.12928587 0.08189561 0.00251147 0.25955449]
9158


In [12]:
micro_stats = get_stats(micro_pred)
print(micro_stats/np.sum(micro_stats))
print(np.sum(micro_stats))

[0.00480688 0.64951931 0.30797774 0.02588969 0.01180638]
11858


In [13]:
pause_stats = get_stats(pause_pred)
print(pause_stats/np.sum(pause_stats))
print(np.sum(pause_stats))

[0.08932397 0.29623899 0.52749345 0.05849798 0.02844561]
16804


In [14]:
rear_stats = get_stats(rear_pred)
print(rear_stats/np.sum(rear_stats))
print(np.sum(rear_stats))

[0.00668281 0.19569849 0.14996936 0.6322993  0.01535005]
34267


In [15]:
walk_stats = get_stats(walk_pred)
print(walk_stats/np.sum(walk_stats))
print(np.sum(walk_stats))

[0.51821279 0.11517296 0.08464361 0.05686583 0.22510482]
7632
